In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import torch
import copy
import torch.nn as nn
import torch.optim as optim
from utils import data_loader, train, eval, nn_modules
from sklearn.preprocessing import MinMaxScaler
from alpha_vantage.timeseries import TimeSeries

%load_ext autoreload
%autoreload 2

In [2]:
API_KEY = 'PZZ6AOYX66Q8H83Q'
ts = TimeSeries(key=API_KEY, output_format='pandas', indexing_type='date')
data, meta_data = ts.get_intraday('FB')
meta_data

{'1. Information': 'Intraday (15min) open, high, low, close prices and volume',
 '2. Symbol': 'FB',
 '3. Last Refreshed': '2019-04-02 14:45:00',
 '4. Interval': '15min',
 '5. Output Size': 'Compact',
 '6. Time Zone': 'US/Eastern'}

In [27]:
batch_size = 1
input_size = 2
hidden_size = 50
num_layers = 2
prediction_window = 3
training_window = 5
dropout=0.2
epochs = 60


dfs = data_loader.load_daily_data('./data/FAANG_13_18_CSV.csv', ['FB'])
x_train, y_train, x_val, y_val = data_loader.create_split(dfs[0], training_window=training_window, prediction_window=prediction_window)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dev_cpu = torch.device("cpu")
net = nn_modules.StockPredictor(input_size, hidden_size, num_layers, device, prediction_window, batch_size=batch_size, p=0.2)
train_itr = data_loader.BatchIterator((x_train, y_train), batch_size)
val_itr = data_loader.BatchIterator((x_val, y_val), batch_size)

loss_function = nn.MSELoss(reduction='sum')
optimizer = optim.Adam(net.parameters(), lr=1e-3)

losses, min_val_mse = train.train(net, train_itr, val_itr, loss_function, optimizer, device, epochs=epochs)


d:\cs 699\env\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Epoch:0, Average loss:0.0017557189567014575
Average val MSE = 0.06250979525755335
Epoch:1, Average loss:0.011914301663637161
Average val MSE = 0.03396828508084147
Epoch:2, Average loss:0.013356568291783333
Average val MSE = 0.031125477118978347
Epoch:3, Average loss:0.01742950826883316
Average val MSE = 0.042122578481006916
Epoch:4, Average loss:0.013159453868865967
Average val MSE = 0.03615282474625985
Epoch:5, Average loss:0.012911141850054264
Average val MSE = 0.02881504224747337
Epoch:6, Average loss:0.025547027587890625
Average val MSE = 0.034255339975016674
Epoch:7, Average loss:0.023514924570918083
Average val MSE = 0.03456373004146959
Epoch:8, Average loss:0.01830807514488697
Average val MSE = 0.03265331343650982
Epoch:9, Average loss:0.014573462307453156
Average val MSE = 0.03364787312026066
Epoch:10, Average loss:0.018339654430747032
Average val MSE = 0.029473675263961288
Epoch:11, Average loss:0.012760069221258163
Average val MSE = 0.031395752727418506
Epoch:12, Average loss

In [28]:
net = eval.load_saved_model('./models/best_model.pt', net)
eval.evaluate_model(net, val_itr)

# net.load_state_dict(torch.load('./models/best_model.pt'))
# net.eval()
# print('Sucessfully loaded model {}'.format('./models/best_model.pt'))

Sucessfully loaded model ./models/best_model.pt
Average test MSE = 0.007376235068984948
